In [1]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
import requests, os
import pandas as pd

In [2]:
# You need to add the infos in the .env file
load_dotenv()

ES_HOST = os.getenv("ES_HOST")
ES_PASSWORD = os.getenv("ES_PASSWORD")
ES_USER = os.getenv("ES_USER")
es = Elasticsearch(ES_HOST, http_auth=(ES_USER, ES_PASSWORD))
INDEX_PROJECTS = f'scanr-projects'
INDEX_PUBLICATIONS = f'scanr-publications'

from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(ES_HOST, http_auth=(ES_USER, ES_PASSWORD))

In [3]:
project_ids = ['ANR-14-FRAL-0003', 'ANR-23-PEIA-0008']

# List of grants, with the associated publications

NB: Rich informations for projects (grants), but just id / title / affiliations for publications (year is missing for now!)

In [4]:
payload = {
    'size': 10000,
    '_source': ['id', 'type', 'year', 'participants', 'call', 'acronym', 'label', 'keywords', 'participantCount', 'description', 'budgetFinanced', 'persons', 'domains', 'action', 'publications', 'publicationsCount'],
    
 'query': {
     'bool': {
         'filter': [
             {"terms": {"id.keyword":project_ids}}
       
   ]}
          }
}
res = es.search(index=INDEX_PROJECTS, request_cache=False, request_timeout=6000, body=payload)
data_grants = [e['_source'] for e in res['hits']['hits']]
df_grants = pd.DataFrame(data_grants)
df_grants

,id,type,year,participants,call,acronym,label,keywords,participantCount,description,budgetFinanced,persons,domains,action,publications,publicationsCount
0,ANR-14-FRAL-0003,ANR,2014,"[{'structure': {'id': '180044083', 'mainAddres...",{},{'default': 'CIBEC'},{'fr': 'Corpus des inscriptions de la Béotie c...,{},2,{'fr': 'Ce projet a pour but la préparation d’...,205987.60,"[{'fullName': 'Yannis KALLIONTZIS', 'firstName...",[],"{'id': 'FRAL', 'label': {'default': 'FRAL'}, '...","[{'id': 'doi10.4000/bch.577', 'projects': ['AN...",1
1,ANR-23-PEIA-0008,PIA ANR,2023,"[{'role': 'participant', 'label': {'default': ...",{},{'default': 'SHARP'},{'fr': 'Sharp Theoretical and Algorithmic Prin...,{},8,{'fr': 'The amount of resources needed to trai...,6390118.46,"[{'fullName': 'Rémi GRIBONVAL', 'firstName': '...","[{'label': {'default': 'ICL'}, 'code': 'Q32806...","{'id': 'PEIA', 'label': {'default': 'PEIA'}, '...","[{'id': 'halhal-04455639', 'projects': ['ANR-1...",41


In [12]:
pd.DataFrame(data_grants[1]['publications']).head()

,id,projects,title,affiliations
0,halhal-04455639,"[ANR-19-P3IA-0001, ANR-23-PEIA-0008]",{'default': 'A PAC-Bayesian Link Between Gener...,"[grid.508487.6, 19350937900015, 491231, https:..."
1,doi10.1109/cvprw63382.2024.00161,[ANR-23-PEIA-0008],{'default': 'What Makes Multimodal In-Context ...,"[grid.462844.8, https://ror.org/02en5vm52, 180..."
2,halhal-04954574,"[ANR-23-CE11-0006, ANR-24-CE23-7039, ANR-19-P3...",{'default': 'Rapture of the deep: highs and lo...,"[300009, anr-ia-23, 180089013, grid.462970.e, ..."
3,halhal-03573458,"[ANR-18-CE40-0016, ANR-23-PEIA-0008]",{'default': 'Controlling Multiple Errors Simul...,"[425779, 0597065J, 374570, 265906719, 18008901..."
4,halhal-05093943,"[ANR-21-CE23-0017, ANR-23-PEIA-0008, ANR-19-CH...",{'default': 'Learning a neural solver for para...,"[grid.4444.0, 200918463J, anr-ia-19, anr, 4415..."


# List of publications, with the associated grants

In [5]:
payload = {
    'size': 10000,
    '_source': ['id', 'projects', 'title', 'summary', 'externalIds', 'year', 'publicationDate', 'type', 'productionType', 'isOa', 'keywords', 'domains', 'affiliations', 'authorsCount', 'authors' ],
 'query': {
     'bool': {
         'filter': [
             {"terms": {"projects.id.keyword":project_ids}}
       
   ]}
          }
}
res = es.search(index=INDEX_PUBLICATIONS, request_cache=False, request_timeout=6000, body=payload)
data_publications = [e['_source'] for e in res['hits']['hits']]
df_publications = pd.DataFrame(data_publications)
df_publications

,id,title,summary,externalIds,year,publicationDate,type,productionType,isOa,keywords,domains,projects,affiliations,authorsCount,authors
0,halhal-04592562,{'default': 'ACCO: Accumulate While You Commun...,{'default': 'Training LLMs relies on distribut...,"[{'type': 'hal', 'id': 'hal-04592562'}]",2025,2025-05-16T00:00:00,other,publication,True,"{'default': ['Distributed Optimization', 'Larg...","[{'label': {'default': 'Reduce'}, 'code': 'Q18...","[{'id': 'ANR-21-CE23-0030', 'label': {'fr': 'O...","[{'id': 'grid.462844.8'}, {'id': 'https://ror....",6,"[{'firstName': 'Adel', 'lastName': 'Nabli', 'f..."
1,halhal-05165202,{'default': 'Interpretable image recognition w...,{'default': 'Prototype networks are intrinsica...,"[{'type': 'hal', 'id': 'hal-05165202'}]",2025,2025-07-16T00:00:00,other,publication,True,"{'default': ['Variable Prototype Allocation', ...",[{'label': {'default': 'Variable Prototype All...,"[{'id': 'ANR-23-PEIA-0008', 'label': {'fr': 'S...","[{'id': '179741'}, {'id': 'grid.36823.3c'}, {'...",3,"[{'firstName': 'Katia', 'lastName': 'Benali', ..."
2,halhal-04785441,{'default': 'Supra-Laplacian Encoding for Tran...,{'default': 'Fully connected Graph Transformer...,"[{'type': 'hal', 'id': 'hal-04785441'}]",2024,2024-10-30T00:00:00,proceedings,publication,True,"{'default': ['Graph Neural Netwoks', 'Dynamic ...","[{'label': {'default': 'Transformers'}, 'code'...","[{'id': 'ANR-23-PEIA-0008', 'label': {'fr': 'S...","[{'id': 'grid.4444.0'}, {'id': '200918463J', '...",4,"[{'firstName': 'Yannis', 'lastName': 'Karmim',..."
3,halhal-04590932,{'default': 'Keep the Momentum: Conservation L...,{'default': 'Conservation laws are well-establ...,"[{'type': 'hal', 'id': 'hal-04590932'}]",2024,2024-05-21T00:00:00,proceedings,publication,True,NaN,"[{'label': {'default': 'Conservation Laws'}, '...","[{'id': 'ANR-19-P3IA-0001', 'label': {'fr': 'P...","[{'id': 'grid.508487.6'}, {'id': '0694094A'}, ...",3,"[{'firstName': 'Sibylle', 'lastName': 'Marcott..."
4,doi10.1007/978-3-031-73013-9_18,{'default': 'Weight Conditioning for Smooth Op...,"{'default': 'In this article, we introduce a n...","[{'type': 'doi', 'id': '10.1007/978-3-031-7301...",2024,2024-11-26T00:00:00,book-chapter,publication,False,NaN,"[{'label': {'default': 'Conditioning'}, 'code'...","[{'id': 'ANR-21-CE23-0017', 'label': {'fr': 'D...","[{'id': 'grid.1010.0', 'mainAddress': {'gps': ...",3,"[{'fullName': 'Hemanth Saratchandran', 'role':..."
5,halhal-04731841,{'default': 'Learning via Surrogate PAC-Bayes'...,{'default': 'PAC-Bayes learning is a comprehen...,"[{'type': 'hal', 'id': 'hal-04731841'}]",2024,2024-10-11T00:00:00,proceedings,publication,True,"{'default': ['Optimisation', 'PAC-Bayes', 'Phy...","[{'label': {'default': 'PAC'}, 'code': 'Q21014...","[{'id': 'ANR-23-PEIA-0008', 'label': {'fr': 'S...","[{'id': 'grid.4444.0'}, {'id': 'grid.83440.3b'...",3,"[{'firstName': 'Antoine', 'lastName': 'Picard-..."
6,doi10.1109/cvprw63382.2024.00161,{'default': 'What Makes Multimodal In-Context ...,{'default': 'Large Language Models have demons...,"[{'type': 'doi', 'id': '10.1109/cvprw63382.202...",2024,2024-06-17T00:00:00,proceedings,publication,True,NaN,"[{'label': {'default': 'ICL'}, 'code': 'Q32806...","[{'id': 'ANR-23-PEIA-0008', 'label': {'fr': 'S...","[{'id': 'grid.462844.8'}, {'id': 'https://ror....",5,"[{'fullName': 'Folco Bertini Baldassini', 'rol..."
7,halhal-04710775,{'default': 'Open-Canopy: Towards Very High Re...,{'default': 'Estimating canopy height and its ...,"[{'type': 'hal', 'id': 'hal-04710775'}]",2025,2025-09-05T00:00:00,proceedings,publication,True,NaN,"[{'label': {'default': 'Canopy'}, 'code': 'Q15...","[{'id': 'ANR-23-PEIA-0008', 'label': {'fr': 'S...","[{'id': 'pasteur_umr8640'}, {'id': 'https://ro...",11,"[{'firstName': 'Fajwel', 'lastName': 'Fogel', ..."
8,halhal-04584311,{'default': 'A Rescaling-Invariant Lipschitz B...,{'default': 'Robustness with respect to weight...,"[{'type': 'hal', 'id': 'hal-04584311'}]",2025,2025-07-13T00:00:00,proceedings,publica